In [96]:
import sys
import os
# Set up TWINT config
import twint
# Solve compatibility issues with notebooks and RunTime errors.
import nest_asyncio
nest_asyncio.apply()
c = twint.Config()

c.Search = "BTC"
# c.Username = "noneprivacy"
# c.Custom["tweet"] = ["id"]
# c.Custom["user"] = ["bio"]

c.Limit = 100
c.Store_pandas = True
# c.Store_csv = True
# c.Output = "test.csv"
c.Limit = 1
c.Pandas = True
twint.run.Search(c)

1487993882201792514 2022-01-30 19:39:21 -0800 <CerdasCrypto> Analis JPMorgan Turunkan Secara Drastis Target Harga Jangka Panjang untuk Bitcoin, Sebutkan Tantangan Terbesar untuk BTC  https://t.co/iuDfARNrWj
1487993876551847940 2022-01-30 19:39:20 -0800 <Thinley16613447> @TheMoonCarl Have been in BTC since 2019.
1487993868335386625 2022-01-30 19:39:18 -0800 <TedatACA> In $BTC?
1487993856537010179 2022-01-30 19:39:15 -0800 <rvxbt_> $BTC  Something like this  https://t.co/1hYegCAWgq
1487993854158585856 2022-01-30 19:39:15 -0800 <Rayshown42> @Political_Jerry @magliani07 @sparrowhead22 @lizelly10 @djbenhameen @MetaMask @Whitehat_BTC #metamaskscam this some balogna how meta gon kick my dawg dog in the face ? @peta
1487993838388097028 2022-01-30 19:39:11 -0800 <SwichLabs> #Bitcoin Risk Metric:  #btc $BTC  https://t.co/b9TzAHcQoN
1487993837947879428 2022-01-30 19:39:11 -0800 <bitcoin_sougo> 世界最大の証券取引所であるニューヨーク証券取引所のCEOが Bitcoinが最初の世界通貨になるだろうと発言 #仮想通貨 $BTC #ビットコイン #BTC #暗号資産 #投資 #ニューヨーク証券取引所 #世

In [108]:
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
# Function to get sentiment 
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative
# UDF to write sentiment on DF
sentiment = udf(apply_blob, DoubleType())
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]
def tweets_sentiment(search, limit=1):
    c.Search = search
    # Custom output format
    c.Format = "Username: {username} |  Tweet: {tweet}"
    c.Limit = limit
    c.Pandas = True
    with HiddenPrints():
        print(twint.run.Search(c))
    
    # Transform tweets to pandas DF
    df_pd = twint_to_pandas(["date", "username", "tweet", "hashtags", "nlikes"])
    
    # Transform Pandas DF to Optimus/Spark DF
    df = op.create.data_frame(pdf= df_pd)
    
    # Clean tweets
    clean_tweets = df.cols.remove_accents("tweet") \
                 .cols.remove_special_chars("tweet")
    
    # Add sentiment to final DF
    return clean_tweets.withColumn("sentiment",    sentiment(clean_tweets['tweet']))

In [110]:
from textblob import TextBlob
from IPython.display import Markdown, display
# Pretty printing the result
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
for tweet in df:
    print(tweet)
    analysis = TextBlob(tweet)
    print(analysis.sentiment)
    if analysis.sentiment[0]>0:
        printmd('Positive', color="green")
    elif analysis.sentiment[0]<0:
        printmd('Negative', color="red")
    else:
        printmd("No result", color="grey")
        print("")

date
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>No result</span>


username
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>No result</span>


tweet
Sentiment(polarity=0.0, subjectivity=0.0)


...

<span style='color:grey'>No result</span>


nretweets
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>No result</span>


nlikes
Sentiment(polarity=0.0, subjectivity=0.0)


<span style='color:grey'>No result</span>

In [67]:
# Transform result to pandas
def twint_to_pandas(columns):
    return twint.output.panda.Tweets_df[columns]

In [80]:
df = twint_to_pandas(["date", "username","tweet", "hashtags", "nreplies", "nretweets","nlikes"])

In [82]:
df.head()

,date,username,tweet,hashtags,nreplies,nretweets,nlikes
0,2022-01-30 13:29:37,saadin_ozdemir,@btc_magazin @gmgiray Savaşami diye srcaktim d...,[],0,0,0
1,2022-01-30 13:29:36,FitzoCrypto,So many influencers last night brought the gid...,"[btc, bitcoin]",0,0,0
2,2022-01-30 13:29:35,yekkripto,#SuddenChange : Coin|From-&gt;To|Last Hour|Tod...,"[suddenchange, pump, crypto, coin]",0,0,0
3,2022-01-30 13:29:34,Vaskar506,"New week, New Top 30! Which of these are in ...",[],0,0,0
4,2022-01-30 13:29:32,Idriss6363,@PessiRakim et est ce que tu es conseillé pour...,[],0,0,0


In [99]:
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        

In [105]:
def get_tweets(search, limit=100):
    c = twint.Config()
    c.Search = search
    c.Limit = limit
    c.Pandas = True
    c.Pandas_clean = True

In [103]:
def generate_word_cloud(tweets):
    
    # Getting the text out of the tweets
    text = tweets.tweet.values
    
    # adding movie script specific stopwords
    stopwords = set(STOPWORDS)
    stopwords.add("https")
    stopwords.add("xa0")
    stopwords.add("xa0'")
    stopwords.add("bitly")
    stopwords.add("bit")
    stopwords.add("ly")
    stopwords.add("twitter")
    stopwords.add("pic")
    
    wordcloud = WordCloud(
        background_color = 'black',
        width = 1000,
        height = 500,
        stopwords = stopwords).generate(str(text))
    
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.rcParams['figure.figsize'] = [10, 10]

In [106]:
tweets = get_tweets("artificial intelligence", limit=1000)
generate_word_cloud(tweets)

AttributeError: 'NoneType' object has no attribute 'tweet'